In [1]:
import os

In [2]:
%pwd

'c:\\Projects\\End-to-End_Cancer_Classification_using_DVC_MLFLOW\\research'

In [3]:
from cnnClassifier.utils.common import save_bin

In [4]:
os.chdir("../")

In [7]:
%pwd

'c:\\Projects\\End-to-End_Cancer_Classification_using_DVC_MLFLOW'

In [8]:
from cnnClassifier.utils.common import save_bin

In [9]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml,create_directories

#### entity


In [26]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig():
    root_dir: Path
    source_URL: Path
    local_data_file: Path
    unzip_dir: Path

#### Configuration.py Code


In [27]:
class ConfigurationManager():
    def __init__(
        self,
        config_file_path= CONFIG_FILE_PATH,
        params_file_path= PARAMS_FILE_PATH):

        self.config= read_yaml(config_file_path)
        self.params= read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self)-> DataIngestionConfig:

        config=self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config=DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir= config.unzip_dir
        )

        return data_ingestion_config


        


#### Data Ingestion Component

In [28]:
from cnnClassifier import logger
import gdown
import zipfile

In [29]:
class DataIngestion:

    def __init__(self,config:DataIngestionConfig):
        self.config= config

    def download_file(self)->str:
        '''fetch Data from url
        '''
        try:
            dataset_url=self.config.source_URL
            zip_download_dir= self.config.local_data_file
            os.makedirs("artifacts/data_ingestion",exist_ok=True)
            logger.info(f"Downloading the dataset:{dataset_url} into file {zip_download_dir}")

            file_id=dataset_url.split("/")[-2]
            prefix = "https://drive.google.com/uc?/export=download&id="
            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"Downloaded Data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
        
    def extract_zip_file(self):
        """
        zip_file_path : str
        Extracts the zip file into the data directory

        """
        unzip_path=self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path)






#### Pipeline 01

In [31]:
try:
    config= ConfigurationManager()
    data_ingestion_config= config.get_data_ingestion_config()
    data_ingestion=DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e


[2024-12-22 21:56:45,926: INFO: common: yaml file:config\config.yaml loaded successfully]
[2024-12-22 21:56:45,929: INFO: common: yaml file:params.yaml loaded successfully]
[2024-12-22 21:56:45,931: INFO: common: created directory:artifacts]
[2024-12-22 21:56:45,932: INFO: common: created directory:artifacts/data_ingestion]
[2024-12-22 21:56:45,932: INFO: 3795941448: Downloading the dataset:https://drive.google.com/file/d/1sgvwfQZPJFRgSyCupYlK8gl4qxc6mdWC/view?usp=sharing into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1sgvwfQZPJFRgSyCupYlK8gl4qxc6mdWC
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1sgvwfQZPJFRgSyCupYlK8gl4qxc6mdWC&confirm=t&uuid=c41e9b47-cf50-4fbc-9682-f516a209e15e
To: c:\Projects\End-to-End_Cancer_Classification_using_DVC_MLFLOW\artifacts\data_ingestion\data.zip
100%|██████████| 49.0M/49.0M [00:14<00:00, 3.39MB/s]

[2024-12-22 21:57:05,097: INFO: 3795941448: Downloaded Data from https://drive.google.com/file/d/1sgvwfQZPJFRgSyCupYlK8gl4qxc6mdWC/view?usp=sharing into file artifacts/data_ingestion/data.zip]
